# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [1]:
import pandas as pd
import numpy as np

# Load the airbnb dataframe
airbnb_df = pd.read_csv('/content/airbnb_hw.csv')

# Clean the 'Price' column
# Remove '$' and ',' characters
airbnb_df['Price'] = airbnb_df['Price'].astype(str).str.replace('$', '').str.replace(',', '')

# Convert to numeric, coercing errors to NaN
airbnb_df['Price'] = pd.to_numeric(airbnb_df['Price'], errors='coerce')

# Count missing values
missing_prices = airbnb_df['Price'].isnull().sum()

print(f"Number of missing values after cleaning: {missing_prices}")

# Display the first few rows with the cleaned 'Price' column
display(airbnb_df[['Price']].head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/airbnb_hw.csv'

In [ ]:
# Load the police use of force dataframe
mn_police_df = pd.read_csv('/content/mn_police_use_of_force.csv')

# Clean the 'subject_injury' column
# Replace values that are not 'Yes' or 'No' with NaN
mn_police_df['subject_injury'] = mn_police_df['subject_injury'].apply(lambda x: x if x in ['Yes', 'No'] else np.nan)

# Count missing values
missing_injury = mn_police_df['subject_injury'].isnull().sum()

print(f"Number of missing values after cleaning: {missing_injury}")

# Find proportion of missing values
proportion_missing = missing_injury / len(mn_police_df)
print(f"Proportion of missing values in subject_injury: {proportion_missing}")

# Display the value counts of the cleaned 'subject_injury' column
display(mn_police_df['subject_injury'].value_counts(dropna=False))

In [ ]:
justice_df = pd.read_parquet('/content/justice_data.parquet')

# Cleaning the 'WhetherDefendantWasReleasedPretrial' column
# Convert to string type first to handle potential mixed types
justice_df['WhetherDefendantWasReleasedPretrial'] = justice_df['WhetherDefendantWasReleasedPretrial'].astype(str)

# Replace any values that are not clearly 'Yes' or 'No' (or similar representations) with NaN
# Assuming 'Yes' and 'No' are the primary valid values, and anything else is considered missing/invalid for this purpose.
# This is a flexible approach depending on what 'cleaning as well as you can' means for this variable.
# If there are specific known invalid strings, they could be explicitly included in a list to replace.
justice_df['WhetherDefendantWasReleasedPretrial'] = justice_df['WhetherDefendantWasReleasedPretrial'].replace(['nan', 'None'], np.nan)

# Display the value counts to see the result of cleaning
display(justice_df['WhetherDefendantWasReleasedPretrial'].value_counts(dropna=False))

# Display the number of missing values
missing_pretrial_release = justice_df['WhetherDefendantWasReleasedPretrial'].isnull().sum()
print(f"Number of mising values after cleaning: {missing_pretrial_release}")


# Clean the missing values for the justice data parquet column ImposedSentenceAllChargeInContactEvent

# Look at both of the columns
display(justice_df['SentenceTypeAllChargesAtConvictionInContactEvent'].value_counts(dropna=False))
display(justice_df['ImposedSentenceAllChargeInContactEvent'].value_counts(dropna=False))

# ImposedSentenceAllChargeInContactEvent is missing when there was no sentence given (so like dismissed or acquitted too)
# Filling the missing values in 'ImposedSentenceAllChargeInContactEvent' with 0 for cases where no sentence was imposed or the case was not a conviction leading to a sentence makes sense

# Making a variable for rows where no sentence was imposed or case was dismissed/acquitted
no_sentence_conditions = [
    justice_df['SentenceTypeAllChargesAtConvictionInContactEvent'].isna(),
    justice_df['SentenceTypeAllChargesAtConvictionInContactEvent'].isin(['Dismissed', 'Acquitted', 'No Sentence Imposed'])]

# Combining these conditions using or concatenation
no_sentence_mask = pd.concat(no_sentence_conditions, axis=1).any(axis=1)

# Filling the missing 'ImposedSentenceAllChargeInContactEvent' values with 0 for these rows
justice_df.loc[no_sentence_mask, 'ImposedSentenceAllChargeInContactEvent'] = justice_df.loc[no_sentence_mask, 'ImposedSentenceAllChargeInContactEvent'].fillna(0)

# For any remaining missing values in 'ImposedSentenceAllChargeInContactEvent', we can assume that they are actually missing values and so I replaced them with np.nan
justice_df['ImposedSentenceAllChargeInContactEvent'] = pd.to_numeric(justice_df['ImposedSentenceAllChargeInContactEvent'], errors='coerce')

# Displaying the value counts and missing values after cleaning
display(justice_df['ImposedSentenceAllChargeInContactEvent'].value_counts(dropna=False))

missing_imposed_sentence = justice_df['ImposedSentenceAllChargeInContactEvent'].isnull().sum()
print(f"\nNumber of missing values after cleaning: {missing_imposed_sentence}")

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)